In [11]:
%load_ext autoreload
%autoreload 2

We define our graph as an igraph.Graph object. Python igraph is a library for high-performance graph generation and analysis. Install the Python library with sudo pip install python-igraph.

Get graph data:

In [12]:
import igraph as ig
import json
import requests
import pickle
import helpers
from bokeh.sampledata.les_mis import data

full_links_dict = pickle.load(open('data/billionairesdict.pkl','rb'))

links_dict = helpers.trim_dict(full_links_dict, 100)
print(links_dict)

data = helpers.dict_to_nodes(links_dict, 'links', 'views')

{'Elon Musk': {'views': 4724522, 'links': ['Bill Gates', 'Larry Page', 'Jeff Bezos', 'Larry Ellison', 'Warren Buffett', 'Peter Thiel', 'Mark Zuckerberg', 'Jack Dorsey', 'Charles Koch']}, 'Donald Trump': {'views': 3208026, 'links': ['Bill Gates', 'Michael Bloomberg', 'Ross Perot', 'Jeff Bezos', 'Larry Ellison', 'Carl Icahn', 'Warren Buffett', 'Vince McMahon', 'Mark Cuban', 'Kanye West', 'Peter Thiel', 'Steve Wynn', 'Jamie Dimon', 'Elon Musk', 'Fred Trump', 'Robert Kraft', 'John Schnatter', 'Tom Monaghan', 'Sheldon Adelson', 'Sheryl Sandberg', 'Betsy DeVos', 'Kenneth C. Griffin', 'Mark Zuckerberg', 'Donald Trump', 'George Soros', 'Dustin Moskovitz', 'Jack Dorsey', 'Oprah Winfrey', 'J. B. Pritzker', 'Terry Pegula', 'Charles Koch', 'Jim Justice', 'Kim Pegula']}, 'Jeff Bezos': {'views': 1670326, 'links': ['Bill Gates', 'Michael Bloomberg', 'Larry Page', 'Jeff Bezos', 'Larry Ellison', 'Warren Buffett', 'David Geffen', 'Phil Knight', 'Alice Walton', 'Elon Musk', 'Sheldon Adelson', 'Kenneth C.

Get the number of nodes:

In [13]:
N=len(data['nodes'])
N

100

Define the list of edges and the Graph object from Edges:

In [14]:
L=len(data['links'])
Edges=[(data['links'][k]['source'], data['links'][k]['target']) for k in range(L)]

G=ig.Graph(Edges, directed=False)

Extract the node attributes, 'group', and 'name':

In [15]:
data['nodes'][0]

{'name': 'Elon Musk', 'group': 0}

In [16]:
labels=[]
group=[]
for node in data['nodes']:
    labels.append(node['name'])
    group.append(node['group'])

Get the node positions, set by the Kamada-Kawai layout for 3D graphs:

In [17]:
layt=G.layout('kk', dim=3)
len(layt)

100

layt is a list of three elements lists (the coordinates of nodes):

In [18]:
layt[0]

[0.8779640215740121, 0.8313875997073297, -0.3483271533418246]

Set data for the Plotly plot of the graph:

In [19]:
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]

In [20]:
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             color=group,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Network of coappearances of characters in Victor Hugo's novel<br> Les Miserables (3D visualization)",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text="Data source: <a href='http://bost.ocks.org/mike/miserables/miserables.json'>[1] miserables.json</a>",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],    )


In [21]:
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

iplot(fig, filename='Les-Miserables')